<a id='top'></a>
# CEERS: Reducing MIRI Imaging Data
---
**Author**: Guang Yang (gyang206265@gmail.com) 

**Latest Update**: 12 November 2021

This notebook will reduce all available raw MIRI images through the JWST Calibration Pipeline, with some custom steps developed for our simulated data. At the end, it also tests the output photometry.

## Table of Contents
* [Imports](#imports)
* [Notebook setup](#setup)
* [Pipeline information](#info)
* [The calwebb_detector1 and calwebb_image2 pipelines](#steps1&2) 
* [The calwebb_image3 pipeline](#step3) 
* [Photometry tests](#photometry)

<a id='imports'></a>
## Imports

In [ ]:
# This cell loads necessary modules 
import numpy as np
import os
from astropy.io import fits

from jwst import pipeline
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3 import Asn_Lv3Image

from glob import glob
import sep
from astroscrappy import detect_cosmics

from astropy.wcs import WCS
from astropy.table import Table

from collections import defaultdict
import astropy.units as u
from astropy.coordinates import SkyCoord
import math
import sys
import matplotlib.pyplot as plt

<a id='setup'></a>
## Set-up

This notebook will reduce all raw data located in `[data_dir]/CEERS*/F*W/`, where `CEERS*` are each individual CEERS MIRI pointing, and the raw files are organized by filter (`F*W`).

**Change the path for `data_dir` in the next cell to match the location of your downloaded CEERS MIRI raw images**

In [ ]:
# This cell set the data directory and search for the raw images

# Set the directory where all the data are
#data_dir = "./"
data_dir = "/home/shared/preloaded-fits/ceers-data/miri/part1"

# Search the work directories hosting raw images
wk_dirs = glob(f"{data_dir}/CEERS*/*W")

print(wk_dirs)

<a id='info'></a>
##  Pipeline information

The cells below will run the [JWST Calibration Pipeline](https://github.com/spacetelescope/jwst). 
We briefly summarize the pipeline below, and more details can be found in the [User Manual](https://jwst-pipeline.readthedocs.io/en/latest/index.html#user-manual).

The inputs are raw MIRI images and the outputs are calibrated science images, weight images, and catalogs (optional; see below).  

The pipeline has three stages: 
* [Stage 1](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1) applies basic detector-level corrections, yielding a count-rate map for each exposure.
* [Stage 2](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#calwebb-image2) applies additional instrumental corrections and calibrations that result in a fully calibrated individual exposure.  
* [Stage 3](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html) combines the calibrated multiple exposures into a single rectified image and perform source detections (if desired). 

Below, we adopt the default settings for Stages 1 & 2 (except skipping a function in Stage 1; see below), but significantly modify Stage 3. The reason is that Stage 3 is still under intensive development and the current default settings cannot produce optimal results for our simulated data (see [Yang et al. 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...908..144Y/abstract) for details). Also, our MIRISIM-simulated raw data do not contain the physical WCS information, and we need to correct the WCS on our own at Stage 3.

<a id='steps1&2'></a>
## The calwebb-detector1 and calwebb_image2 pipelines

Converting ramps to slopes ([*calwebb_detector1*, Detector1Pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1)) and producing calibrated slope images ([*calwebb_image2*, Image2Pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image2.html)). 

We will call the pipeline using the [`run()` method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

This cell will take several minutes to run, depending on how many raw images you are reducing. The Stage 1 reduction pipeline steps in particular can take a while for each input raw image.

Note: We skip the "refpix" function in Stage 1, because MIRISIM dose not put in the targeted noise of this function and running it would cause stripe-like artifacts in the final science images. 

In [ ]:
# This cell performs stages 1 & 2 reduction

#Option to skip stage 1 if preprocessed data already exists on the system
#Use this to save time for JWebbinar instructional purposes
skip_stage_1 = True
#skip_stage_1 = False

# Iterater over each band
for wk_dir in wk_dirs:
    # Search for raw files 
    raw_files = glob(os.path.join(wk_dir,'*exp?.fits'))
    # The list to save the path of stage1 products 
    stage1_files = []
    # Iterate over the input
    det1 = Detector1Pipeline()
    # Skip the refpix function
    det1.refpix.skip = True
    print(raw_files)
    for file in raw_files:

        # Generate the file name of stage1 product
        stage1_bn = os.path.basename(file[:file.rfind('.fits')] + '_stage1.fits')
        stage1_fil = os.path.basename(os.path.dirname(file))
        stage1_relpath = os.path.join('CEERS7b',stage1_fil)
        os.makedirs(stage1_relpath,exist_ok=True)
        stage1_file = os.path.join(stage1_relpath,stage1_bn)
        
        #If pre-staged files exist, link to them from the local directory.
        if skip_stage_1 is True:
            if not os.path.exists(stage1_file):
                os.symlink(os.path.join(wk_dir,stage1_bn),stage1_file)
        
        #check if stage 1 files exist already.  If not, run stage 1.
        if os.path.lexists(stage1_file):
            print('stage 1 output exists, skipping..', stage1_file)
            stage1_files.append(stage1_file)
        else:
            # Run the pipeline
            result = det1.run(file)
        
            # Save the stage1 products to file
            result.save(stage1_file)
            # Save the file name
            stage1_files.append(stage1_file)

    # Process to stage 2
    #stage1_files = glob(wk_dir+'*stage1*.fits')
    print(stage1_files)
    stage2_files = []
    # Iterate over the input
    for file in stage1_files:
        # Run the pipeline
        img2 = Image2Pipeline()
        result = img2.run(file)
        # Generate the file name of stage2 product
        stage2_bn = os.path.basename(file[:file.rfind('stage1.fits')] + 'stage2.fits')
        stage2_fil = os.path.basename(os.path.dirname(file))
        stage2_relpath = os.path.join('CEERS7b',stage2_fil)
        os.makedirs(stage2_relpath,exist_ok=True)
        stage2_file = os.path.join(stage2_relpath,stage2_bn)
        # Save the results to file
        result[0].save(stage2_file)
        # Save the file name
        stage2_files.append(stage2_file)   

[Top of Notebook](#top)

### Redefine work dirs to local path

In [ ]:
print(stage2_files)
wk_dirs = glob(f"CEERS*/*W")
print(wk_dirs)

<a id='step3'></a>
## The calwebb_image3 pipeline

Ensemble calibrations ([*calwebb_image3*, Image3Pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image3.html))

In [ ]:
# This cell performs Stage 3 reduction, 
# correct for WCS (only needed for simulated images), and 
# perform photometry  

# Set below to True if you want the pipeline to extract photometry using photutils
photo_flag = True

# Set the background configurations
bkg_cell_size = 16
bkg_filt_size = 3
fthresh = -99

# The correction factors for different bands 
# These are obtained by simulating bright point-like sources
# (see Appendix A of Yang+2021)
cor_facs = {"F560W":1.069, 
            "F770W":1.156, 
            "F1000W":1.040, 
            "F1280W":0.998, 
            "F1500W":1.062, 
            "F1800W":0.973, 
            "F2100W":1.047}

# CR maps (produced by gen_cr_map.ipynb) needed below
for wk_dir in wk_dirs:
    # Process to stage 3
    stage2_files = sorted( glob(os.path.join(wk_dir,'*stage2.fits')) )
    # Create an association 
    asn = dict( asn_from_list.asn_from_list(stage2_files, rule=Asn_Lv3Image, product_name='l3_results') )
    asn["asn_type"] = "image3"
    asn["asn_id"] = ""
    # Configure the stage 3 pipeline 
    img3 = Image3Pipeline()
    img3.output_dir = wk_dir
    img3.output_file = 'stage3_merged_image'
    img3.resample.save_results = False
    img3.resample.output_dir = wk_dir

    # Run the pipeline
    res_step3 = img3.skymatch.run(asn)
        
    # Subtract the position-dependent backgrounds
    for seq_idx in range(len(res_step3._models)):
        # Extract the data
        sci_img = np.array(res_step3._models[seq_idx].data)
        dq_img = np.array(res_step3._models[seq_idx].dq)
        err_img = np.array(res_step3._models[seq_idx].err)
        bkg_level = res_step3._models[seq_idx].meta.background.level
        # Mask bad pixels
        mask = (dq_img>100)
        mask[sci_img < bkg_level/2] = True

        # Also mask the nearby pixels
        bad_idxs = np.where(mask)
        # To avoid index underflow
        use_Sidxs = np.where( bad_idxs[0]!=0 )
        mask[bad_idxs[0][use_Sidxs]-1, bad_idxs[1][use_Sidxs]] = True
        use_Sidxs = np.where( bad_idxs[1]!=0 )
        mask[bad_idxs[0][use_Sidxs], bad_idxs[1][use_Sidxs]-1] = True
        # To avoid index overflow 
        use_Sidxs = np.where( bad_idxs[0]!=sci_img.shape[0]-1 )
        mask[bad_idxs[0][use_Sidxs]+1, bad_idxs[1][use_Sidxs]] = True
        use_Sidxs = np.where( bad_idxs[1]!=sci_img.shape[1]-1 )
        mask[bad_idxs[0][use_Sidxs], bad_idxs[1][use_Sidxs]+1] = True
        
        # Mask coronograph
        mask[:, :350] = True
        # Estimate background
        bkg = sep.Background(sci_img.byteswap().newbyteorder(), 
                             bw=bkg_cell_size, bh=bkg_cell_size, 
                             fw=bkg_filt_size, fh=bkg_filt_size, mask=mask, fthresh=fthresh)
        # Subtract background
        sci_img -= bkg.back()
        # Re-set the metadata, indicating background already subtracted
        res_step3._models[seq_idx].meta.background.subtracted = True
        # Mask defect pixels
        mask[sci_img < -10*err_img] = True
        # Mask the CR 
        cr_map = detect_cosmics(sci_img, sigclip=1)[0]*1        
        mask[np.where(cr_map)] = True
        
        # Set masked region to zero
        sci_img[mask] = 0
        # Update the data
        res_step3._models[seq_idx].data = sci_img
        # Update dq
        res_step3._models[seq_idx].dq[mask] = np.max(res_step3._models[seq_idx].dq)     

    # Merge the exposures
    res_step4 = img3.resample.run(res_step3)
    mask = res_step4.wht==0
    # subtract the background
    bkg = sep.Background(res_step4.data, 
                         bw=bkg_cell_size, bh=bkg_cell_size, 
                         fw=bkg_filt_size, fh=bkg_filt_size, mask=mask, fthresh=fthresh)
    res_step4.data -= bkg.back()
    res_step4.data[mask] = 0
    res_step4.save(os.path.join(wk_dir,'stage3_merged_image_resample.fits'))
    
    # Updte WCS and output final science and weight images
    # Read mirisim products
    mirisim_data = fits.open(os.path.join(wk_dir,'stage3_merged_image_resample.fits'))[1]
    hd = mirisim_data.header
    # Extract the old wcs info.
    old_wcs = WCS(hd)

    # Read the pointing info.
    pt_id = wk_dir[wk_dir.find('CEERS')+5:wk_dir.find('CEERS')+7]
    if pt_id[1]=='/': pt_id=pt_id[0]
    reg_data = Table.read(os.path.join(data_dir,"pointing_info/miri%s_reg.fits" %pt_id))[0]
    # Get the new reference pixel
    new_crpix_1, new_crpix_2 = old_wcs.all_world2pix([0], [0], 1)
    new_crpix_1, new_crpix_2 = new_crpix_1[0], new_crpix_2[0]
    # Re-set the ra-dec for the new pix ref
    new_crval_1, new_crval_2 = reg_data['RA_Ref'], reg_data['DEC_Ref']
    # Update the rotation matrix
    theta = reg_data['angle']*np.pi/180 + np.arcsin(mirisim_data.header['PC1_2'])
    new_pc1_1, new_pc1_2 = -np.cos(theta), np.sin(theta)
    new_pc2_1, new_pc2_2 =  np.sin(theta), np.cos(theta)
    # Fill in the new WCS and write to file
    mirisim_data.header['CRPIX1'] = new_crpix_1
    mirisim_data.header['CRPIX2'] = new_crpix_2
    mirisim_data.header['CRVAL1'] = new_crval_1
    mirisim_data.header['CRVAL2'] = new_crval_2
    mirisim_data.header['PC1_1'] = new_pc1_1
    mirisim_data.header['PC1_2'] = new_pc1_2
    mirisim_data.header['PC2_1'] = new_pc2_1
    mirisim_data.header['PC2_2'] = new_pc2_2
        
    # Apply corrections 
    filt = wk_dir[wk_dir.find('F'):wk_dir.find('W')+1]
    mirisim_data.data *= cor_facs[filt]
    # Write to file
    fits.writeto(wk_dir+'final_image.fits', mirisim_data.data, 
                 header=mirisim_data.header, overwrite=True)
    # Write the weight image
    wht_data = fits.open(os.path.join(wk_dir,'stage3_merged_image_resample.fits'))[3]
    fits.writeto(os.path.join(wk_dir,'final_weight.fits'), wht_data.data, 
                 header=mirisim_data.header, overwrite=True)
    
    # Extract photometry (if needed)
    if photo_flag:
        # Do not save results immediately, as we need to correct WCS
        img3.source_catalog.save_results = False
        # Perform source detection and photometry extraction 
        # Parameters can be changed by img3.source_catalog.update_pars
        # Use img3.source_catalog.get_pars to display current parameters
        res_step5 = img3.source_catalog.run(res_step4)
        # Correct for WCS
        # Read the corrected WCS
        cor_wcs = WCS(mirisim_data.header)
        # Transform x,y to ra,dec
        ras, decs = cor_wcs.all_pix2world(res_step5['xcentroid'], res_step5['ycentroid'], 0)
        # Update ra,dec in res_step5 
        for src_idx, _ in enumerate(res_step5):
            res_step5['sky_centroid'][src_idx] = \
                SkyCoord(ras[src_idx],decs[src_idx], unit='deg')
        # Save the catalog to file
        res_step5.write(os.path.join(wk_dir,'photutils_catalog.fits'), overwrite=True)

[Top of Notebook](#top)

<a id='photometry'></a>
## Photometry tests

The cells below are an example test, i.e., measured photometry vs. model-input photometry. This example test shows apparent magnitude offsets. This is expected, because the photutils is not fine-tuned when performing photometry (see above). 

In [ ]:
# This cell define a function to match between two catalogs 

def match(ra1, de1, ra2, de2, rad, opt = 2, silent=False):
    '''
    This function will perform one-one match between two catalogs
    
    Inputs: 
        ra1, de1, ra2, de2: coordiantes in two catalogs, should be array-like unit: deg
        rad: matching radius, unit: arcsec

    Keywords:
        opt: same as srcor.pro in IDL
                0, source in catalog 2 may correspond to >1 sources in catalog 1
                1, forced 1-1 match 
                2, forced 1-1 match, but the program will run two times. At the first run is to obtain the 
                   median coordinate offset between the two catalogs. Then we apply the offset to
                   one catalog to eliminate the systematic difference. The second run is based on the
                   updated coordinates. (default) 
        silent: True, do not display results for opt=0 or 1
                False, display results (default)

    Outputs:
        id1, indexes of matched sources in catalog I 
        id2, indexes of matched sources in catalog II
        dis, distances of the pairs, unit: arcsec 
    '''

    # Format unification
    ra1 = np.array(ra1)
    de1 = np.array(de1)
    ra2 = np.array(ra2)
    de2 = np.array(de2)

    # Check if all RA, DEC are float
    if ( np.where(np.isfinite(ra1))[0].size != ra1.size ) | \
       ( np.where(np.isfinite(de1))[0].size != de1.size ) | \
       ( np.where(np.isfinite(ra2))[0].size != ra2.size ) | \
       ( np.where(np.isfinite(de2))[0].size != de2.size ) : \
        sys.exit('Error: non-float in ra,dec')

    if ra1.shape != de1.shape:
        raise ValueError('ra1 and dec1 do not match!')
    if ra2.shape != de2.shape:
        raise ValueError('ra2 and dec2 do not match!')

    # Initial match, note that id_2 may contain repetitive elements
    if opt == 0:
        id1, id2, dis = _ini_match(ra1, de1, ra2, de2, rad/3600.)
        if not(silent):
            print('Multi-one match: ', len(id1), ' sources')
        return id1, id2, dis*3600.

    # First match, only the nearest neighbor is left when multi-to-one occurs
    id1, id2, dis = _1st_match(ra1, de1, ra2, de2, rad/3600.)
    if opt == 1:
        if not(silent):
            print('Forced one-one match: ', len(id1), ' sources')
        return id1, id2, dis*3600.
    print('First match: ', len(id1), ' sources')

    # Second match, elliminate the offset
    print( 'RA offset:', \
           np.median(ra1[id1] - ra2[id2]) * 3600 \
           * math.cos(np.deg2rad(np.median(de1))), 'arcsec', \
           'DEC offset:', \
           np.median(de1[id1] - de2[id2])*3600, 'arcsec')
    ra2_new = ra2 + np.median(ra1[id1] - ra2[id2])
    de2_new = de2 + np.median(de1[id1] - de2[id2])
    id1, id2, dis = _1st_match(ra1, de1, ra2_new, de2_new, rad/3600.)
    print('Second match: ', len(id1), ' sources')

    return id1, id2, dis*3600
            

def _1st_match(ra1, de1, ra2, de2, rad):
    '''
    This function performs the forced 1-1 match. Similar to srcor.pro (opt=1) in IDL.
    
    Inputs: 
        ra1, de1, ra2, de2: coordiantes in two catalogs, unit: deg
        rad: matching radius, unit: deg
    
    Outputs:
        id1, indexes of matched sources in catalog I 
        id2, indexes of matched sources in catalog II
        dis, distances of the pairs, unit: deg 
    '''

    # Initial match, might produce multi-one match
    id1, id2, dis = _ini_match(ra1, de1, ra2, de2, rad)

    # Find the repetitive ones  (routine)
    rep = defaultdict(list)
    for i, item in enumerate(id2):
        rep[item].append(i)
    rep = {k:v for k,v in rep.items() if len(v) > 1}
    # note that k is indexes in catalog II, v is indexes in the matching array

    # For them, reserve the nearest one only
    kill = []
    for dum, idx in rep.items(): 
        dis_rep = dis[idx] # Their distances 
        best = dis_rep.argmin()  # Pick up the nearest one
        kill.append(np.delete(idx, best)) # Creat the to-be-delete list

    buf = np.arange(0) # Transform format from list to numpy array
    while kill != []: 
        buf = np.append(buf, kill.pop())
    kill = buf
        
    id1 = np.delete(id1, kill) # Remove them
    id2 = np.delete(id2, kill)
    dis = np.delete(dis, kill) 

    return id1, id2, dis


def _ini_match(ra1, de1, ra2, de2, rad):
    '''
    This function is modified from pyspherematch.py, available at
    https://gist.github.com/eteq/4599814
    The main change is, turn to match_coordinates_sky (astropy) from KDTree (scipy).
    Because the second is based on 'straight-line' distance rather than 
    'great-circle' distance.

    Inputs: 
        ra1, de1, ra2, de2: coordiantes in two catalogs, unit: deg
        rad: matching radius, unit: deg
    
    Outputs:
        id1, indexes of matched sources in catalog I 
        id2, indexes of matched sources in catalog II
        dis, distances of the pairs, unit: deg 
    '''

    cor1 = SkyCoord(ra=ra1*u.deg, dec=de1*u.deg, frame='icrs') # Change to ICRS system
    cor2 = SkyCoord(ra=ra2*u.deg, dec=de2*u.deg, frame='icrs')

    id2, dis, buf = cor1.match_to_catalog_sky(cor2) # Match
    dis = np.array(dis) # Drop the unit
    id1 = np.arange(ra1.size)
    if id2.size<=1:
        id2 = np.array([id2])
    else:
        id2 = np.array(id2)

    msk = dis < rad	 # Filter out those with distance > rad
    id1 = id1[msk]
    id2 = id2[msk]
    dis = dis[msk]

    return id1, id2, dis

The next cell will compare the input and measure photometry for a given CEERS MIRI pointing and band. 

**Set the pointing ID and band you want to compare in the first two lines (`pt_id` and `band`)**.

In [ ]:
# This cell makes a plot comparing measured vs. input photometry

# Set the pointing ID and the band you want to compare 
pt_id = '7b'
band = 'F770W'

# Read the input the measured photometry
input_cat = Table.read(os.path.join(data_dir,'model_cat/miri%s_src_sam.fits' %pt_id))
meas_cat = Table.read(os.path.join('.','CEERS%s/%s/photutils_catalog.fits' %(pt_id, band)))

# Extract ra & dec from photutils results
phot_ras, phot_decs = [], []
for row in meas_cat:
    phot_ras.append(row['sky_centroid'].ra.deg)
    phot_decs.append(row['sky_centroid'].dec.deg)
# Match photutils and input sources 
input_phot_idxs, phot_input_idxs, ds = match(input_cat['ra'], input_cat['dec'], phot_ras, phot_decs, 1, opt=2)
# Compare the magnitude 
input_mags = -2.5*np.log10(input_cat[input_phot_idxs][band])+23.9
phot_mags = meas_cat['aper_total_abmag'][phot_input_idxs]
# Plot mag. vs. mag
plt.figure(figsize=(6,8))
plt.subplot(211)
plt.plot(input_mags, phot_mags, 'ko', alpha=0.5)
plt.plot(range(31),'C3')
plt.xlim(np.min(input_mags)*0.95, np.max(input_mags)*1.05)
plt.ylim(np.min(input_mags)*0.95, np.max(input_mags)*1.05)
#plt.xlim(20, 26)
#plt.ylim(20, 26)
plt.xlabel('model mag')
plt.ylabel('photutils mag')
# Plot det_mag vs. mag
plt.subplot(212)
plt.plot(input_mags, phot_mags-input_mags, 'ko', alpha=0.5)
plt.plot([0,30],[0,0],'C3')
plt.xlim(np.min(input_mags)*0.95, np.max(input_mags)*1.05)
#plt.xlim(20, 26)
plt.ylim(-2, 2)
plt.xlabel('model mag')
plt.ylabel(r'$\Delta$mag')

[Top of Notebook](#top)